### Escenario 1: Algoritmos de aprendizaje profundo

En esta sección analizaremos algoritmos de aprendizaje profundo. Se plantea el uso de un algoritmo DNN con los tres escenarios.

In [1]:
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import layers
from keras import models
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, confusion_matrix

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#data_esc1 = pd.read_csv("/content/drive/MyDrive/Colab_Tesis/Churn_Modelling_escenario1.csv")
data_esc2 = pd.read_csv("datasets/Churn_Modelling_escenario1.csv")

In [4]:
data_esc2.head(3)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_encoded,Gender_encoded
0,619,42,2,0.00,1,1,1,101348.88,1,2,1
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,2,1


In [5]:
data_esc2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             10000 non-null  int64  
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Exited             10000 non-null  int64  
 9   Geography_encoded  10000 non-null  int64  
 10  Gender_encoded     10000 non-null  int64  
dtypes: float64(2), int64(9)
memory usage: 859.5 KB


In [6]:
# División del dataset en entrenamiento y prueba

features = data_esc2.drop("Exited", axis=1)
target = data_esc2["Exited"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

#### 2 Balanceo de Datos con Oversampling y Undersampling

##### 2.1. Balanceo de Datos con Oversampling (SMOTE)

In [7]:
smote_esc2 = SMOTE(random_state=42)
X_train_over, y_train_over = smote_esc2.fit_resample(X_train, y_train)

print('Distribución de clases antes del oversampling:', Counter(y_train))
print('Distribución de clases después del oversampling:', Counter(y_train_over))

Distribución de clases antes del oversampling: Counter({0: 5547, 1: 1453})
Distribución de clases después del oversampling: Counter({1: 5547, 0: 5547})


##### 2.2 Balanceo de Datos con Undersampling

In [8]:
rus_esc2 = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = rus_esc2.fit_resample(X_train, y_train)

#### 3 Estandarización de los Datos

In [9]:
# Se realiza el proceso de estandarización de las características (features)
escalado = StandardScaler()

# Se estandariza el conjunto de datos original (desbalanceado)
X_train_std_esc2 = escalado.fit_transform(X_train)
X_test_std_esc2 = escalado.transform(X_test)

# Se estandariza el conjunto de datos con Oversampling
X_train_over_std_esc2 = escalado.fit_transform(X_train_over)
X_test_over_std_esc2 = escalado.transform(X_test)

# Se estandariza el conjunto de datos con Undersampling
X_train_under_std_esc2 = escalado.fit_transform(X_train_under)
X_test_under_std_esc2 = escalado.transform(X_test)

#### 4 Entrenamiento y Evaluación del Modelo

##### 4.1. Aplicamos el modelo de una Red Neuronal Profunda (DNN)

In [10]:
# Definir y entrenar el modelo DNN
model = keras.Sequential()
model.add(layers.Dense(64, input_dim=X_train_std_esc2.shape[1], activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Compilación y entrenamiento
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_std_esc2, y_train, epochs=10, batch_size=16, validation_split=0.2)

# Evaluación
y_pred = model.predict(X_test_std_esc2)
y_pred = (y_pred > 0.5)  # Convertir probabilidades en etiquetas
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Epoch 1/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7487 - loss: 0.5328 - val_accuracy: 0.8379 - val_loss: 0.4007
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8525 - loss: 0.3796 - val_accuracy: 0.8443 - val_loss: 0.3621
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8522 - loss: 0.3554 - val_accuracy: 0.8464 - val_loss: 0.3653
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8573 - loss: 0.3570 - val_accuracy: 0.8493 - val_loss: 0.3544
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8624 - loss: 0.3384 - val_accuracy: 0.8471 - val_loss: 0.3517
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8575 - loss: 0.3472 - val_accuracy: 0.8443 - val_loss: 0.3518
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8593 - loss: 0.3364 - val_accuracy: 0.8507 - val_loss: 0.3480
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8669 - loss: 0.3220 - val_accuracy: 0.

##### 4.2. Aplicamos el modelo de una Red Neuronal Profunda (DNN) con Oversampling

In [12]:
# Definir y entrenar el modelo DNN
model = keras.Sequential()
model.add(layers.Dense(64, input_dim=X_train_over_std_esc2.shape[1], activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#model = keras.Sequential(
#    [
#        layers.Dense(64, input_dim=X_train_over_std_esc1.shape[1], activation='relu'),
#        layers.Dense(32, activation='relu'),
#        layers.Dense(16, activation='relu'),
#        layers.Dense(1, activation='sigmoid')
#    ]
#)

c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Definir y entrenar el modelo DNN
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_over_std_esc2, y_train_over, epochs=10, batch_size=16, validation_split=0.2)

# Se realiza la evaluación del modelo
y_pred_over = model.predict(X_test_over_std_esc2)
y_pred_over = (y_pred_over > 0.5)

print(confusion_matrix(y_test, y_pred_over))
print(classification_report(y_test, y_pred_over))

Epoch 1/10
555/555 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7000 - loss: 0.5559 - val_accuracy: 0.7639 - val_loss: 0.4713
Epoch 2/10
555/555 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8013 - loss: 0.4365 - val_accuracy: 0.7688 - val_loss: 0.4727
Epoch 3/10
555/555 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8073 - loss: 0.4173 - val_accuracy: 0.8026 - val_loss: 0.4096
Epoch 4/10
555/555 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8176 - loss: 0.3986 - val_accuracy: 0.8333 - val_loss: 0.3635
Epoch 5/10
555/555 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8316 - loss: 0.3840 - val_accuracy: 0.7999 - val_loss: 0.4346
Epoch 6/10
555/555 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8238 - loss: 0.3908 - val_accuracy: 0.7512 - val_loss: 0.5132
Epoch 7/10
555/555 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8273 - loss: 0.3868 - val_accuracy: 0.7679 - val_loss: 0.4887
Epoch 8/10
555/555 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8318 - loss: 0.3803 - val_accuracy: 0.

##### 4.3. Aplicamos el modelo de una Red Neuronal Profunda (DNN) con Undersampling

In [14]:
# Definir y entrenar el modelo DNN
model = keras.Sequential()
model.add(layers.Dense(64, input_dim=X_train_under_std_esc2.shape[1], activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Definir y entrenar el modelo DNN
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_under_std_esc2, y_train_under, epochs=10, batch_size=16, validation_split=0.2)

# Evaluación
y_pred_under = model.predict(X_test_under_std_esc2)
y_pred_under = (y_pred_under > 0.5)

print(confusion_matrix(y_test, y_pred_under))
print(classification_report(y_test, y_pred_under))

Epoch 1/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6280 - loss: 0.6308 - val_accuracy: 0.4691 - val_loss: 0.8544
Epoch 2/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7277 - loss: 0.5399 - val_accuracy: 0.6340 - val_loss: 0.7249
Epoch 3/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7716 - loss: 0.4864 - val_accuracy: 0.6512 - val_loss: 0.7000
Epoch 4/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7731 - loss: 0.4623 - val_accuracy: 0.6289 - val_loss: 0.6530
Epoch 5/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7829 - loss: 0.4464 - val_accuracy: 0.6048 - val_loss: 0.7584
Epoch 6/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7909 - loss: 0.4388 - val_accuracy: 0.5979 - val_loss: 0.7512
Epoch 7/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8041 - loss: 0.4262 - val_accuracy: 0.6959 - val_loss: 0.6553
Epoch 8/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8114 - loss: 0.4165 - val_accuracy: 0.

##### 5.1. Aplicamos el modelo de una Red Neuronal Convolucional (CNN)

In [16]:
# Remodelar los datos para adaptarlos a una CNN (1 canal, porque los datos son tabulares)
X_train_cnn_std = X_train_std_esc2.reshape(X_train_std_esc2.shape[0], X_train_std_esc2.shape[1], 1)
X_test_cnn_std = X_test_std_esc2.reshape(X_test_std_esc2.shape[0], X_test_std_esc2.shape[1], 1)

# Crear el modelo CNN
model = models.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=(X_train_cnn_std.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 8, 64)          │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 2, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,841 (140.00 KB)

 Trainable params: 35,841 (140.00 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Entrenar el modelo
model.fit(X_train_cnn_std, y_train, epochs=10, batch_size=16, validation_data=(X_test_cnn_std, y_test))

# Evaluación del modelo con datos completos
y_pred_cnn = (model.predict(X_test_cnn_std) > 0.5)
print("Evaluación con datos completos:")
print(classification_report(y_test, y_pred_cnn))
print(confusion_matrix(y_test, y_pred_cnn))

Epoch 1/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7913 - loss: 0.4956 - val_accuracy: 0.8307 - val_loss: 0.3966
Epoch 2/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8185 - loss: 0.4118 - val_accuracy: 0.8413 - val_loss: 0.3802
Epoch 3/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8378 - loss: 0.3905 - val_accuracy: 0.8517 - val_loss: 0.3691
Epoch 4/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8422 - loss: 0.3795 - val_accuracy: 0.8520 - val_loss: 0.3783
Epoch 5/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8538 - loss: 0.3557 - val_accuracy: 0.8590 - val_loss: 0.3552
Epoch 6/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8508 - loss: 0.3577 - val_accuracy: 0.8540 - val_loss: 0.3603
Epoch 7/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8552 - loss: 0.3469 - val_accuracy: 0.8610 - val_loss: 0.3513
Epoch 8/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8531 - loss: 0.3483 - val_accuracy: 0.

In [18]:
print(classification_report(y_test, y_pred_cnn))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      2416
           1       0.69      0.48      0.57       584

    accuracy                           0.86      3000
   macro avg       0.79      0.72      0.74      3000
weighted avg       0.85      0.86      0.85      3000



##### 5.2. Aplicamos el modelo de una Red Neuronal Profunda (CNN) con Oversampling

In [19]:
# Remodelar para CNN con Oversampling
X_train_cnn_std_over = X_train_over_std_esc2.reshape(X_train_over_std_esc2.shape[0], X_train_over_std_esc2.shape[1], 1)
X_test_cnn_std_over = X_test_over_std_esc2.reshape(X_test_over_std_esc2.shape[0], X_test_over_std_esc2.shape[1], 1)

# Crear el modelo CNN
model = models.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=(X_train_cnn_std_over.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 8, 64)          │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 2, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,841 (140.00 KB)

 Trainable params: 35,841 (140.00 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Entrenar el modelo con datos balanceados con Oversampling
model.fit(X_train_cnn_std_over, y_train_over, epochs=10, batch_size=16, validation_data=(X_test_cnn_std_over, y_test))

# Evaluar el modelo
# Evaluación del modelo con datos completos
y_pred_cnn_over = (model.predict(X_test_cnn_std_over) > 0.5)
print("Evaluación con datos completos:")
print(classification_report(y_test, y_pred_cnn_over))
print(confusion_matrix(y_test, y_pred_cnn_over))

Epoch 1/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7273 - loss: 0.5433 - val_accuracy: 0.7883 - val_loss: 0.4494
Epoch 2/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8015 - loss: 0.4437 - val_accuracy: 0.8370 - val_loss: 0.3835
Epoch 3/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8041 - loss: 0.4179 - val_accuracy: 0.8187 - val_loss: 0.4016
Epoch 4/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8183 - loss: 0.4025 - val_accuracy: 0.7977 - val_loss: 0.4610
Epoch 5/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8214 - loss: 0.3926 - val_accuracy: 0.7970 - val_loss: 0.4475
Epoch 6/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8276 - loss: 0.3861 - val_accuracy: 0.7667 - val_loss: 0.5028
Epoch 7/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8299 - loss: 0.3846 - val_accuracy: 0.7930 - val_loss: 0.4416
Epoch 8/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8315 - loss: 0.3743 - val_accuracy: 0.

##### 5.3. Aplicamos el modelo de una Red Neuronal convolucional (CNN) con undersampling

In [21]:
# Remodelar para CNN con Undersampling
X_train_cnn_std_under = X_train_under_std_esc2.reshape(X_train_under_std_esc2.shape[0], X_train_under_std_esc2.shape[1], 1)
X_test_cnn_std_under = X_test_under_std_esc2.reshape(X_test_under_std_esc2.shape[0], X_test_under_std_esc2.shape[1], 1)

# Crear el modelo CNN
model = models.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=(X_train_cnn_std_under.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 8, 64)          │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 2, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,841 (140.00 KB)

 Trainable params: 35,841 (140.00 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Entrenar el modelo con datos balanceados con Undersampling
model.fit(X_train_cnn_std_under, y_train_under, epochs=10, batch_size=16, validation_data=(X_test_cnn_std_under, y_test))

# Evaluar el modelo
# Evaluación del modelo con datos completos
y_pred_cnn_under = (model.predict(X_test_cnn_std_under) > 0.5)
print("Evaluación con datos completos:")
print(classification_report(y_test, y_pred_cnn_under))
print(confusion_matrix(y_test, y_pred_cnn_under))

Epoch 1/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5830 - loss: 0.6579 - val_accuracy: 0.7417 - val_loss: 0.5157
Epoch 2/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6909 - loss: 0.5779 - val_accuracy: 0.7453 - val_loss: 0.5039
Epoch 3/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7199 - loss: 0.5409 - val_accuracy: 0.6590 - val_loss: 0.5638
Epoch 4/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7474 - loss: 0.5166 - val_accuracy: 0.5427 - val_loss: 0.7406
Epoch 5/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7347 - loss: 0.5097 - val_accuracy: 0.6157 - val_loss: 0.6534
Epoch 6/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7687 - loss: 0.4932 - val_accuracy: 0.6943 - val_loss: 0.5860
Epoch 7/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7691 - loss: 0.4631 - val_accuracy: 0.7763 - val_loss: 0.4737
Epoch 8/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7472 - loss: 0.4971 - val_accuracy: 0.

##### 6.1. Aplicamos el modelo de una Red Neuronal recurrente (LSTM)

In [23]:
# Se remodela para que sea compatible con LSTM (n_samples, n_features, 1)
X_train_lstm_std = X_train_std_esc2.reshape(X_train_std_esc2.shape[0], X_train_std_esc2.shape[1], 1)
X_test_lstm_std = X_test_std_esc2.reshape(X_test_std_esc2.shape[0], X_test_std_esc2.shape[1], 1)
# Ver la nueva forma de los datos
print(X_train_lstm_std.shape, X_test_lstm_std.shape)

# Crear el modelo LSTM
model = models.Sequential([
    layers.LSTM(64, activation='relu', input_shape=(X_train_lstm_std.shape[1], 1)),
    layers.Dropout(0.2),  # Regularización por Dropout
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Salida binaria (0 o 1)
])
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

(7000, 10, 1) (3000, 10, 1)


c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,681 (92.50 KB)

 Trainable params: 23,681 (92.50 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Entrenar el modelo
model.fit(X_train_lstm_std, y_train, epochs=10, batch_size=16, validation_data=(X_test_lstm_std, y_test))

# Evaluar el modelo
y_pred_lstm = (model.predict(X_test_lstm_std) > 0.5)
print("Evaluación con datos completos:")
print(classification_report(y_test, y_pred_lstm))
print(confusion_matrix(y_test, y_pred_lstm))

Epoch 1/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7931 - loss: 0.5376 - val_accuracy: 0.8053 - val_loss: 0.4816
Epoch 2/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7989 - loss: 0.4756 - val_accuracy: 0.8173 - val_loss: 0.4335
Epoch 3/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7999 - loss: 0.4508 - val_accuracy: 0.8327 - val_loss: 0.4054
Epoch 4/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8209 - loss: 0.4258 - val_accuracy: 0.8363 - val_loss: 0.4020
Epoch 5/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8138 - loss: 0.4243 - val_accuracy: 0.8413 - val_loss: 0.3890
Epoch 6/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8122 - loss: 0.4329 - val_accuracy: 0.8403 - val_loss: 0.3886
Epoch 7/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8232 - loss: 0.4105 - val_accuracy: 0.8350 - val_loss: 0.3940
Epoch 8/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8228 - loss: 0.4081 - val_accuracy: 0.

##### 6.2. Aplicamos el modelo de una Red Neuronal recurrente (LSTM) con oversampling

In [25]:
# Remodelar para LSTM con Oversampling
X_train_lstm_std_over = X_train_over_std_esc2.reshape(X_train_over_std_esc2.shape[0], X_train_over_std_esc2.shape[1], 1)
X_test_lstm_std_over = X_test_over_std_esc2.reshape(X_test_over_std_esc2.shape[0], X_test_over_std_esc2.shape[1], 1)

#Crear el modelo LSTM
model = models.Sequential([
    layers.LSTM(64, activation='relu', input_shape=(X_train_lstm_std_over.shape[1], 1)),
    layers.Dropout(0.2),  # Regularización por Dropout
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Salida binaria (0 o 1)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,681 (92.50 KB)

 Trainable params: 23,681 (92.50 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Entrenar el modelo con datos balanceados con Oversampling
model.fit(X_train_lstm_std_over, y_train_over, epochs=10, batch_size=16, validation_data=(X_test_lstm_std_over, y_test))

# Evaluar el modelo
y_pred_lstm_over = (model.predict(X_test_lstm_std_over) > 0.5)
print("Evaluación con datos completos:")
print(classification_report(y_test, y_pred_lstm_over))
print(confusion_matrix(y_test, y_pred_lstm_over))

Epoch 1/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6476 - loss: 0.6356 - val_accuracy: 0.7360 - val_loss: 0.5383
Epoch 2/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7321 - loss: 0.5456 - val_accuracy: 0.7073 - val_loss: 0.5894
Epoch 3/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7547 - loss: 0.5094 - val_accuracy: 0.7243 - val_loss: 0.5485
Epoch 4/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7662 - loss: 0.4961 - val_accuracy: 0.7920 - val_loss: 0.4566
Epoch 5/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7820 - loss: 0.4637 - val_accuracy: 0.6863 - val_loss: 0.6025
Epoch 6/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7826 - loss: 0.4590 - val_accuracy: 0.7707 - val_loss: 0.4661
Epoch 7/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7838 - loss: 0.4576 - val_accuracy: 0.7807 - val_loss: 0.4607
Epoch 8/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7943 - loss: 0.4429 - val_accuracy: 0.

##### 6.3. Aplicamos el modelo de una Red Neuronal recurrente (LSTM) con undersampling

In [27]:
# Remodelar para LSTM con Undersampling
X_train_lstm_std_under = X_train_under_std_esc2.reshape(X_train_under_std_esc2.shape[0], X_train_under_std_esc2.shape[1], 1)
X_test_lstm_std_under = X_test_under_std_esc2.reshape(X_test_under_std_esc2.shape[0], X_test_under_std_esc2.shape[1], 1)

#Crear el modelo LSTM
model = models.Sequential([
    layers.LSTM(64, activation='relu', input_shape=(X_train_lstm_std_under.shape[1], 1)),
    layers.Dropout(0.2),  # Regularización por Dropout
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Salida binaria (0 o 1)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,681 (92.50 KB)

 Trainable params: 23,681 (92.50 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Entrenar el modelo con datos balanceados con Undersampling
model.fit(X_train_lstm_std_under, y_train_under, epochs=10, batch_size=16, validation_data=(X_test_lstm_std_under, y_test))

# Evaluar el modelo
y_pred_lstm_under = (model.predict(X_test_lstm_std_under) > 0.5)
print("Evaluación con datos completos:")
print(classification_report(y_test, y_pred_lstm_under))
print(confusion_matrix(y_test, y_pred_lstm_under))

Epoch 1/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.5039 - loss: 0.6940 - val_accuracy: 0.1943 - val_loss: 0.7130
Epoch 2/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.4885 - loss: 0.6936 - val_accuracy: 0.7987 - val_loss: 0.6702
Epoch 3/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5625 - loss: 0.6800 - val_accuracy: 0.7183 - val_loss: 0.6324
Epoch 4/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6106 - loss: 0.6487 - val_accuracy: 0.6490 - val_loss: 0.6522
Epoch 5/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6468 - loss: 0.6293 - val_accuracy: 0.7310 - val_loss: 0.5800
Epoch 6/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6548 - loss: 0.6224 - val_accuracy: 0.5617 - val_loss: 0.7351
Epoch 7/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6658 - loss: 0.6023 - val_accuracy: 0.7717 - val_loss: 0.5103
Epoch 8/10
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6914 - loss: 0.5782 - val_accuracy: 

#### Construcción de la Red Neuronal CNN + Bidirectional LSTM

In [29]:
def build_cnn_bilstm_model(input_shape):
    model = keras.Sequential()

    # Capa convolucional 1
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))

    # Capa convolucional 2
    model.add(layers.Conv1D(128, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))

    # Capa Bidirectional LSTM
    model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
    model.add(layers.Dropout(0.5))

    # Aplanar la salida
    model.add(layers.Flatten())

    # Capa densa
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))

    # Capa de salida
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compilación del modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [30]:
# Reshape para datos 1D, para alimentar a la CNN + LSTM
X_train_BI_std = X_train_std_esc2.reshape(X_train_std_esc2.shape[0], X_train_std_esc2.shape[1], 1)
X_test_BI_std = X_test_std_esc2.reshape(X_test_std_esc2.shape[0], X_test_std_esc2.shape[1], 1)

# Construir el modelo
model = build_cnn_bilstm_model(X_train_BI_std.shape[1:])

c:\Users\rdcastillo47\Documents\RAHAM_UNIVERSIDAD\HERRAMIENTAS\appSIFIA_copia\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
# Entrenar el modelo
model.fit(X_train_BI_std, y_train, epochs=10, batch_size=16, validation_data=(X_test_BI_std, y_test))
# Evaluación del modelo con datos desbalanceados
y_pred_BI = (model.predict(X_test_BI_std) > 0.5)
print("Evaluación con datos desbalanceados:")
print(classification_report(y_test, y_pred_BI))
print(confusion_matrix(y_test, y_pred_BI))

Epoch 1/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.7981 - loss: 0.5189 - val_accuracy: 0.8267 - val_loss: 0.4086
Epoch 2/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8195 - loss: 0.4210 - val_accuracy: 0.8280 - val_loss: 0.3871
Epoch 3/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8275 - loss: 0.4113 - val_accuracy: 0.8520 - val_loss: 0.3671
Epoch 4/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8449 - loss: 0.3855 - val_accuracy: 0.8557 - val_loss: 0.3566
Epoch 5/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8535 - loss: 0.3676 - val_accuracy: 0.8490 - val_loss: 0.3701
Epoch 6/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8442 - loss: 0.3865 - val_accuracy: 0.8617 - val_loss: 0.3489
Epoch 7/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8585 - loss: 0.3600 - val_accuracy: 0.8613 - val_loss: 0.3444
Epoch 8/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8537 - loss: 0.3586 - val_accuracy: 0.

In [32]:
print(classification_report(y_test, y_pred_BI))

              precision    recall  f1-score   support

           0       0.85      0.99      0.92      2416
           1       0.86      0.30      0.45       584

    accuracy                           0.85      3000
   macro avg       0.86      0.65      0.68      3000
weighted avg       0.86      0.85      0.83      3000

